## Monkey Patching

Python allows developers to patch functions, classes, modules at runtime using technique called `Monkey patching`, without actually updating them. It can be used for all types of attributes.

It is very useful for

- debugging your code
- It replaces `methods`/`attributes`/`functions` at runtime which can be used for unit-testing
- Modifying/extending behavior of third-party product without changing the original source code

In the below example, we have `dummy_func` which at runtime we are populating with custom `defaults` dictionary. 

In [1]:
def dummy_func(key):
    print(locals())
    print("defaults" in globals())
    if locals().get("defaults", None):
        return dummy_func.defaults.get(key, "dummy")
    else:
        return None


print(dummy_func({}))
dummy_func.defaults = {"ipaddr": "127.0.0.1", "ip6addr": "::1"}

print(dummy_func("ipaddr"))

{'key': {}}
False
None
{'key': 'ipaddr'}
False
None


### Class methods

For monkey patching class methods, We call the `Class_Name.method = new_method` to make sure that now all calls to `object.method` will

In [2]:
class OriginalClass(object):
    def join(self, x, y):
        return x + y
    
    
oc = OriginalClass()
print(oc.join(10, 20))

30


In [3]:
def joining_list(self, a, b):
    a.extend(b)
    return a


def testing_list(a, b):
    OriginalClass.join = joining_list
    oc = OriginalClass()
    return oc.join(a, b)

print(testing_list([2, 3], [4, 5]))

[2, 3, 4, 5]


#### Gotcha's

Now, that `OriginalClass` has been patched. Any new instance creation will have access to patched version only and the original version is not accessible

In [4]:
oc = OriginalClass()
try:
    print(oc.join(10, 20))
except AttributeError as ae:
    print(ae)

'int' object has no attribute 'extend'


We can avoid the above, by initially creating a variable which is pointing to the original function.

In [5]:
class OriginalClass(object):
    def join(self, x, y):
        return x + y
    
    
oc = OriginalClass()

print(oc.join(10, 20))
ori_join = OriginalClass.join

# Lets patch 
def joining_list(self, a, b):
    a.extend(b)
    return a

def testing_list(a, b):
    OriginalClass.join = joining_list
    oc = OriginalClass()
    return oc.join(a, b)

print(testing_list([2, 3], [4, 5]))

30
[2, 3, 4, 5]


In [6]:
try:
    print(oc.join(10, 20))
except AttributeError as ae:
    print(ae)

'int' object has no attribute 'extend'


In [7]:
## Now everything should work ;). 

try:
    # unpatching it :)
    OriginalClass.join = ori_join
    print(oc.join(10, 20))
except AttributeError as ae:
    print(ae)

30


### References

- https://en.wikipedia.org/wiki/Monkey_patch
    